# 使用Functional API构建复杂模型

本教程介绍Keras的Functional API，它比Sequential API更加灵活，
可以构建非线性拓扑结构、共享层和多输入/多输出模型。

## 学习目标

1. 理解Functional API的语法和工作原理
2. 掌握多输入模型的构建方法
3. 学会构建多输出模型（辅助输出）
4. 理解Wide & Deep架构的设计理念

## Functional API vs Sequential API

| 特性 | Sequential API | Functional API |
|------|---------------|----------------|
| 拓扑结构 | 线性堆叠 | 任意有向无环图 |
| 多输入/多输出 | 不支持 | 支持 |
| 层共享 | 不支持 | 支持 |
| 适用场景 | 简单模型 | 复杂架构 |

## 1. 环境配置与数据准备

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 设置随机种子
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

print(f"TensorFlow版本: {tf.__version__}")

In [ ]:
# 加载并预处理数据
housing = fetch_california_housing()

# 划分数据集
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, test_size=0.2, random_state=RANDOM_SEED
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=0.25, random_state=RANDOM_SEED
)

# 标准化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

print(f"训练集: {X_train.shape}")
print(f"验证集: {X_valid.shape}")
print(f"测试集: {X_test.shape}")
print(f"特征数: {X_train.shape[1]}")

## 2. Functional API基础

### 基本语法

Functional API将层视为函数，通过函数调用的方式连接层：

```python
# 创建输入层
inputs = keras.layers.Input(shape=(n_features,))

# 将层作为函数调用，传入上一层的输出
hidden = keras.layers.Dense(30, activation='relu')(inputs)
outputs = keras.layers.Dense(1)(hidden)

# 创建模型，指定输入和输出
model = keras.Model(inputs=inputs, outputs=outputs)
```

In [ ]:
# 使用Functional API构建简单模型

# 第一步：创建输入层
# Input层定义了输入数据的形状
input_layer = keras.layers.Input(shape=X_train.shape[1:], name='input')

# 第二步：通过函数调用连接各层
# 每个Dense层返回一个张量，可以作为下一层的输入
hidden1 = keras.layers.Dense(30, activation='relu', name='hidden1')(input_layer)
hidden2 = keras.layers.Dense(30, activation='relu', name='hidden2')(hidden1)
output_layer = keras.layers.Dense(1, name='output')(hidden2)

# 第三步：创建Model对象
model_basic = keras.Model(inputs=input_layer, outputs=output_layer, name='basic_functional')

model_basic.summary()

## 3. Wide & Deep模型

### 架构介绍

Wide & Deep模型由Google在2016年提出，结合了：
- **Wide部分**: 线性模型，擅长记忆（memorization）特征组合
- **Deep部分**: 深度神经网络，擅长泛化（generalization）

### 架构图

```
         输入特征
        /        \
       /          \
    Wide路径     Deep路径
      |           |
      |        Hidden1
      |           |
      |        Hidden2
       \          /
        \        /
         Concatenate
             |
           输出
```

In [ ]:
# 构建Wide & Deep模型 - 单输入版本

# 输入层
input_layer = keras.layers.Input(shape=X_train.shape[1:], name='input')

# Deep路径：通过多层非线性变换学习复杂特征
hidden1 = keras.layers.Dense(30, activation='relu', name='deep_hidden1')(input_layer)
hidden2 = keras.layers.Dense(30, activation='relu', name='deep_hidden2')(hidden1)

# Wide路径：直接将原始输入与Deep路径输出连接
# 这使得模型可以同时学习复杂模式和简单线性关系
concat = keras.layers.Concatenate(name='concat')([input_layer, hidden2])

# 输出层
output_layer = keras.layers.Dense(1, name='output')(concat)

# 创建模型
model_wide_deep = keras.Model(inputs=input_layer, outputs=output_layer, name='wide_and_deep')

model_wide_deep.summary()

In [ ]:
# 编译并训练Wide & Deep模型
model_wide_deep.compile(
    loss='mse',
    optimizer=keras.optimizers.SGD(learning_rate=1e-3),
    metrics=['mae']
)

history = model_wide_deep.fit(
    X_train, y_train,
    epochs=30,
    validation_data=(X_valid, y_valid),
    verbose=1
)

## 4. 多输入模型

在实际应用中，我们可能希望将不同类型的特征分别送入不同的网络路径。

### 示例场景

将California Housing数据集的8个特征分为两组：
- **Wide输入 (5个特征)**: 前5个特征直接送入Wide路径
- **Deep输入 (6个特征)**: 后6个特征送入Deep路径

注意：某些特征可能同时出现在两个输入中（特征2-4重叠）

In [ ]:
# 准备多输入数据
# Wide输入：特征0-4（共5个）
# Deep输入：特征2-7（共6个）

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]

print(f"Wide输入形状: {X_train_A.shape}")
print(f"Deep输入形状: {X_train_B.shape}")

In [ ]:
# 构建多输入Wide & Deep模型

# 定义两个输入
input_A = keras.layers.Input(shape=[5], name='wide_input')
input_B = keras.layers.Input(shape=[6], name='deep_input')

# Deep路径：处理deep_input
hidden1 = keras.layers.Dense(30, activation='relu', name='deep_hidden1')(input_B)
hidden2 = keras.layers.Dense(30, activation='relu', name='deep_hidden2')(hidden1)

# 合并Wide和Deep路径
concat = keras.layers.Concatenate(name='concat')([input_A, hidden2])

# 输出层
output = keras.layers.Dense(1, name='output')(concat)

# 创建模型：指定多个输入
model_multi_input = keras.Model(
    inputs=[input_A, input_B], 
    outputs=output,
    name='multi_input_wide_deep'
)

model_multi_input.summary()

In [ ]:
# 编译多输入模型
model_multi_input.compile(
    loss='mse',
    optimizer=keras.optimizers.SGD(learning_rate=1e-3),
    metrics=['mae']
)

# 训练时传入元组或列表形式的多输入数据
history_multi = model_multi_input.fit(
    (X_train_A, X_train_B), y_train,      # 或使用 [X_train_A, X_train_B]
    epochs=30,
    validation_data=((X_valid_A, X_valid_B), y_valid),
    verbose=1
)

In [ ]:
# 评估多输入模型
test_loss, test_mae = model_multi_input.evaluate(
    (X_test_A, X_test_B), y_test, verbose=0
)
print(f"测试集 MSE: {test_loss:.4f}")
print(f"测试集 MAE: {test_mae:.4f}")

# 使用模型预测
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]
y_pred = model_multi_input.predict((X_new_A, X_new_B), verbose=0)
print(f"\n预测值: {y_pred.flatten()}")
print(f"实际值: {y_test[:3]}")

## 5. 多输出模型（辅助输出）

### 设计理念

在深度网络中添加辅助输出有以下好处：

1. **正则化效果**: 确保底层网络学到有用的特征
2. **梯度流动**: 为网络提供额外的梯度信号
3. **多任务学习**: 同时优化多个相关目标

### 架构设计

```
    Wide输入      Deep输入
        \           |
         \       Hidden1
          \         |
           \     Hidden2 -----> 辅助输出
            \       /
           Concatenate
                |
            主输出
```

In [ ]:
# 构建多输入多输出模型

# 输入层
input_A = keras.layers.Input(shape=[5], name='wide_input')
input_B = keras.layers.Input(shape=[6], name='deep_input')

# Deep路径
hidden1 = keras.layers.Dense(30, activation='relu', name='deep_hidden1')(input_B)
hidden2 = keras.layers.Dense(30, activation='relu', name='deep_hidden2')(hidden1)

# 合并层
concat = keras.layers.Concatenate(name='concat')([input_A, hidden2])

# 主输出：基于合并后的特征
main_output = keras.layers.Dense(1, name='main_output')(concat)

# 辅助输出：直接从Deep路径的hidden2层输出
# 这个输出帮助确保hidden2学到了有用的特征表示
aux_output = keras.layers.Dense(1, name='aux_output')(hidden2)

# 创建模型：指定多个输出
model_multi_output = keras.Model(
    inputs=[input_A, input_B],
    outputs=[main_output, aux_output],
    name='multi_output_wide_deep'
)

model_multi_output.summary()

In [ ]:
# 编译多输出模型
# 为每个输出指定损失函数和权重
model_multi_output.compile(
    loss=['mse', 'mse'],           # 两个输出都使用MSE
    loss_weights=[0.9, 0.1],       # 主输出权重0.9，辅助输出权重0.1
    optimizer=keras.optimizers.SGD(learning_rate=1e-3),
    metrics=['mae']
)

# 训练时需要为每个输出提供标签
# 这里两个输出预测相同的目标（房价）
history_multi_output = model_multi_output.fit(
    (X_train_A, X_train_B), 
    (y_train, y_train),            # 两个输出使用相同的标签
    epochs=30,
    validation_data=((X_valid_A, X_valid_B), (y_valid, y_valid)),
    verbose=1
)

In [ ]:
# 评估多输出模型
results = model_multi_output.evaluate(
    (X_test_A, X_test_B), (y_test, y_test), verbose=0
)

print("评估结果:")
print(f"总损失: {results[0]:.4f}")
print(f"主输出损失: {results[1]:.4f}")
print(f"辅助输出损失: {results[2]:.4f}")
print(f"主输出MAE: {results[3]:.4f}")
print(f"辅助输出MAE: {results[4]:.4f}")

In [ ]:
# 多输出预测
y_pred_main, y_pred_aux = model_multi_output.predict(
    (X_test_A[:5], X_test_B[:5]), verbose=0
)

print("预测结果对比:")
print("="*50)
for i in range(5):
    print(f"样本{i+1}: 实际={y_test[i]:.3f}, "
          f"主预测={y_pred_main[i][0]:.3f}, "
          f"辅助预测={y_pred_aux[i][0]:.3f}")

## 6. 可视化训练过程

In [ ]:
# 绘制多输出模型的训练曲线
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 主输出损失
axes[0].plot(history_multi_output.history['main_output_loss'], label='Train Main')
axes[0].plot(history_multi_output.history['val_main_output_loss'], label='Val Main')
axes[0].plot(history_multi_output.history['aux_output_loss'], label='Train Aux', linestyle='--')
axes[0].plot(history_multi_output.history['val_aux_output_loss'], label='Val Aux', linestyle='--')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Loss Curves')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 总损失
axes[1].plot(history_multi_output.history['loss'], label='Train Total')
axes[1].plot(history_multi_output.history['val_loss'], label='Val Total')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Total Loss')
axes[1].set_title('Total Loss (Weighted)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 小结

### Functional API的核心特点

1. **灵活性**: 支持任意的层连接方式
2. **多输入**: 可以接收不同类型的输入数据
3. **多输出**: 可以同时产生多个预测结果
4. **层共享**: 同一层可以被多次调用

### Wide & Deep架构的应用场景

- **推荐系统**: 结合用户历史行为（记忆）和用户特征（泛化）
- **CTR预测**: 同时建模稀疏特征交叉和密集特征
- **结构化数据**: 当需要同时学习简单规则和复杂模式时

### 辅助输出的使用建议

- 辅助输出权重通常设置较小（0.1-0.3）
- 主要用于训练时的正则化，推理时可忽略
- 适用于深度网络，帮助解决梯度消失问题